# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-09-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-09-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-04-10 04:20:59,33.93911,67.709953,57019,2521,51961.0,2537.0,Afghanistan,146.471693,4.421333
1,NaN,NaN,NaN,Albania,2021-04-10 04:20:59,41.15330,20.168300,127795,2304,96672.0,28819.0,Albania,4440.718604,1.802887
2,NaN,NaN,NaN,Algeria,2021-04-10 04:20:59,28.03390,1.659600,118251,3123,82392.0,32736.0,Algeria,269.665194,2.640992
3,NaN,NaN,NaN,Andorra,2021-04-10 04:20:59,42.50630,1.521800,12456,120,11732.0,604.0,Andorra,16121.141526,0.963391
4,NaN,NaN,NaN,Angola,2021-04-10 04:20:59,-11.20270,17.873900,23242,549,21589.0,1104.0,Angola,70.716882,2.362103


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56454,56517,56572,56595,56676,56717,56779,56873,56943,57019
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,125157,125506,125842,126183,126531,126795,126936,127192,127509,127795
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,117192,117304,117429,117524,117622,117739,117879,118004,118116,118251


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2484,2489,2495,2496,2497,2508,2512,2512,2516,2521
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2235,2241,2247,2256,2265,2274,2283,2291,2297,2304
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3093,3096,3099,3102,3105,3108,3112,3116,3119,3123


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,51550,51788,51798,51802,51885,51902,51928,51940,51956,51961
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,91271,91875,92500,93173,93842,94431,95035,95600,96129,96672
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,81538,81632,81729,81813,81896,81994,82096,82192,82289,82392


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6589,6595,6606,6617,6619,6620,6644,6675,6702,6710
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20505,20523,20519,20526,20541,20542,20551,20573,20588,20600
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2227,2227,2228,2231,2232,2232,2238,2239,2244,2245


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,99,99,99,102,102,102,102,102,103,103
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,301,301,301,301,301,301,301,301,300,300
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,55,55,55,55,55,55,55,55,55,55


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-04-10 04:20:59,32.539527,-86.644082,6710,103,NaN,NaN,"Autauga, Alabama, US",12010.238236,1.535022
689,1075.0,Lamar,Alabama,US,2021-04-10 04:20:59,33.779950,-88.096680,1392,34,NaN,NaN,"Lamar, Alabama, US",10083.303151,2.442529
690,1077.0,Lauderdale,Alabama,US,2021-04-10 04:20:59,34.901719,-87.656247,9196,227,NaN,NaN,"Lauderdale, Alabama, US",9917.070172,2.468465


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,31084065,561058,0.0
Brazil,13373174,348718,11722181.0
India,13060542,167642,11913292.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,31084065,561058,0.0,30523007.0,2021-04-10 04:20:59,37.950547,-91.419547
Brazil,13373174,348718,11722181.0,1302275.0,2021-04-10 04:20:59,-12.669522,-48.480493
India,13060542,167642,11913292.0,979608.0,2021-04-10 04:20:59,23.088275,81.806127
France,5000314,98202,312540.0,4589572.0,2021-04-10 04:20:59,6.430808,-34.730285
Russia,4572053,100554,4200282.0,271217.0,2021-04-10 04:20:59,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3692777,60270,0.0
Texas,2822889,49121,0.0
Florida,2111807,33968,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3692777,60270,0.0,3632507.0,2021-04-10 04:20:59,37.843962,-120.728594
Texas,2822889,49121,0.0,2773768.0,2021-04-10 04:20:59,31.660643,-98.653069
Florida,2111807,33968,0.0,2077839.0,2021-04-10 04:20:59,28.940755,-82.700744
New York,1942469,50977,0.0,1891492.0,2021-04-10 04:20:59,42.544151,-75.474183
Illinois,1273085,23762,0.0,1249323.0,2021-04-10 04:20:59,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1224503,23431,0.0
Arizona,Maricopa,527566,9716,0.0
Illinois,Cook,510006,9746,0.0
Florida,Miami-Dade,457918,5954,0.0
Texas,Harris,381664,5955,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1224503,23431,0.0,1201072.0,2021-04-10 04:20:59,34.308284,-118.228241,6037.0
Arizona,Maricopa,527566,9716,0.0,517850.0,2021-04-10 04:20:59,33.348359,-112.491815,4013.0
Illinois,Cook,510006,9746,0.0,500260.0,2021-04-10 04:20:59,41.841448,-87.816588,17031.0
Florida,Miami-Dade,457918,5954,0.0,451964.0,2021-04-10 04:20:59,25.611236,-80.551706,12086.0
Texas,Harris,381664,5955,0.0,375709.0,2021-04-10 04:20:59,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,56873,127192,118004,12363,23010,1177,2450068,198898,29385,566008,...,4381830,126987,84127,3,170189,2659,259133,5047,89386,36984
2021-04-08,56943,127509,118116,12409,23108,1180,2473751,200129,29390,568914,...,4384954,130657,84322,3,171373,2668,262017,5133,89592,37052
2021-04-09,57019,127795,118251,12456,23242,1182,2497881,201158,29396,571616,...,4380167,137946,84529,3,172461,2683,264395,5233,89783,37147


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,2512,2291,3116,119,547,29,56832,3647,909,9546,...,127171,1231,633,0,1705,35,2753,986,1224,1531
2021-04-08,2516,2297,3119,120,549,29,57122,3675,909,9586,...,127224,1275,634,0,1720,35,2781,1004,1225,1532
2021-04-09,2521,2304,3123,120,549,30,57350,3697,909,9624,...,127284,1363,634,0,1739,35,2812,1022,1226,1535


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,51940,95600,82192,11616,21545,911,2174625,178227,23056,522774,...,13391,98837,82055,1,154023,2429,226090,1886,85409,34803
2021-04-08,51956,96129,82289,11692,21557,913,2188983,179043,23071,525682,...,13435,101241,82234,1,154811,2429,227988,1946,85446,34814
2021-04-09,51961,96672,82392,11732,21589,913,2203926,180160,23080,529191,...,13467,104241,82415,1,155662,2429,229876,1987,85559,34831


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6589,6595,6606,6617,6619,6620,6644,6675,6702,6710
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1384,1390,1392,1392,1392,1392,1392,1392,1392,1392
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9140,9144,9148,9156,9159,9162,9165,9178,9188,9196
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2926,2928,2931,2932,2932,2932,2938,2947,2950,2950
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15417,15433,15444,15452,15457,15459,15468,15468,15480,15491


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-07,517452,64303,0,846230,331505,3687493,472168,318767,97042,49,...,819505,2815110,388426,20442,2945,629155,372170,144374,641642,56700
2021-04-08,517916,64531,0,846900,331704,3690031,474053,319779,97404,49,...,820965,2819600,388909,20669,2971,631083,373212,144820,642757,56802
2021-04-09,518234,64782,0,848202,331882,3692777,475989,321586,97784,49,...,822085,2822889,389353,20815,2971,632625,374794,145299,643951,56873


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-04-07        6675   20573    2239  2557   6469    1211   2116   14251   
2021-04-08        6702   20588    2244  2560   6472    1213   2119   14263   
2021-04-09        6710   20600    2245  2561   6475    1213   2118   14270   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-07         3491     1816  ...    2646    611     3083      724   
2021-04-08         3494     1818  ...    2650    611     3085      731   
2021-04-09         3496     1818  ...    2653    611     3085      734   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-07           4067  3645  2140          0      892    639  
2021-04-08           4079  3653  2140          0      893    639  
2021-04-09           4085  3660  2142          0      897    640  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-07,10652,313,0,17023,5660,59993,6140,7935,1567,0,...,11976,48918,2139,230,26,10415,5306,2722,7356,701
2021-04-08,10675,313,0,17055,5662,60129,6143,7940,1568,0,...,11997,49017,2149,230,26,10436,5316,2735,7370,701
2021-04-09,10686,313,0,17062,5661,60270,6151,7944,1573,0,...,12001,49121,2157,231,26,10451,5322,2737,7375,701


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-07         102     301      55   58    133      40     66     310   
2021-04-08         103     300      55   58    134      40     66     310   
2021-04-09         103     300      55   58    134      40     66     311   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-07          121       43  ...      30     11       31        7   
2021-04-08          122       43  ...      30     11       31        7   
2021-04-09          122       43  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-07             37     9    12          0       26      5  
2021-04-08             37     9    12          0       26      5  
2021-04-09             37     9    12          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,0.001656,0.002017,0.001060,0.002839,0.005462,0.003410,0.009077,0.005180,0.000204,0.005509,...,0.000639,0.031886,0.002287,0.0,0.006595,0.004154,0.010419,0.014472,0.003537,0.000487
2021-04-08,0.001231,0.002492,0.000949,0.003721,0.004259,0.002549,0.009666,0.006189,0.000170,0.005134,...,0.000713,0.028901,0.002318,0.0,0.006957,0.003385,0.011129,0.017040,0.002305,0.001839
2021-04-09,0.001335,0.002243,0.001143,0.003788,0.005799,0.001695,0.009754,0.005142,0.000204,0.004749,...,-0.001092,0.055787,0.002455,0.0,0.006349,0.005622,0.009076,0.019482,0.002132,0.002564


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,0.000000,0.003504,0.001285,0.017094,0.000000,0.000000,0.003496,0.005514,0.0,0.003047,...,0.000354,0.033585,0.001582,NaN,0.007088,0.0,0.006581,0.010246,0.000000,0.000000
2021-04-08,0.001592,0.002619,0.000963,0.008403,0.003656,0.000000,0.005103,0.007678,0.0,0.004190,...,0.000417,0.035743,0.001580,NaN,0.008798,0.0,0.010171,0.018256,0.000817,0.000653
2021-04-09,0.001987,0.003047,0.001282,0.000000,0.000000,0.034483,0.003991,0.005986,0.0,0.003964,...,0.000472,0.069020,0.000000,NaN,0.011047,0.0,0.011147,0.017928,0.000816,0.001958


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,0.000231,0.005945,0.001169,0.003976,0.002606,0.029379,0.004889,0.003525,0.000304,0.006880,...,0.003447,0.023549,0.002125,0.0,0.006272,0.00289,0.006849,0.035126,0.000832,0.000633
2021-04-08,0.000308,0.005533,0.001180,0.006543,0.000557,0.002195,0.006603,0.004578,0.000651,0.005563,...,0.003286,0.024323,0.002181,0.0,0.005116,0.00000,0.008395,0.031813,0.000433,0.000316
2021-04-09,0.000096,0.005649,0.001252,0.003421,0.001484,0.000000,0.006826,0.006239,0.000390,0.006675,...,0.002382,0.029632,0.002201,0.0,0.005497,0.00000,0.008281,0.021069,0.001322,0.000488


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-07,0.000654,0.003464,NaN,0.000887,0.000737,0.000664,0.004070,0.003267,0.002438,0.0,...,0.001830,0.001366,0.001578,0.003387,0.000000,0.002470,0.004095,0.002528,0.001464,0.001431
2021-04-08,0.000897,0.003546,NaN,0.000792,0.000600,0.000688,0.003992,0.003175,0.003730,0.0,...,0.001782,0.001595,0.001243,0.011105,0.008829,0.003064,0.002800,0.003089,0.001738,0.001799
2021-04-09,0.000614,0.003890,NaN,0.001537,0.000537,0.000744,0.004084,0.005651,0.003901,0.0,...,0.001364,0.001166,0.001142,0.007064,0.000000,0.002443,0.004239,0.003308,0.001858,0.001250


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-07      0.004666  0.001071  0.000447  0.003138  0.001083  0.001654   
2021-04-08      0.004045  0.000729  0.002233  0.001173  0.000464  0.001652   
2021-04-09      0.001194  0.000583  0.000446  0.000391  0.000464  0.000000   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2021-04-07      0.001420  0.000421 -0.000859  0.001103  ...  0.000756    0.0   
2021-04-08      0.001418  0.000842  0.000859  0.001101  ...  0.001512    0.0   
2021-04-09     -0.000472  0.000491  0.000572  0.000000  ...  0.001132    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2021-04-07     -0.000324  0.001383   0.001724  0.001374  0.002342        NaN   
2021-04-08      0.000649  0.009669   0.002951  0.002195  0.000000        NaN   
2021-04-09      0.000000  0.004104   0.001471  0.001916  0.000935        NaN   

Province_State                      
Admin2          Washakie    Weston  
2021-04-07      0.000000  0.006299  
2021-04-08      0.001121  0.000000  
2021-04-09      0.004479  0.001565  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-07,0.000000,0.0,NaN,0.001589,0.001238,0.001770,0.001305,0.000631,0.000000,NaN,...,0.000752,0.002110,0.000936,0.004367,0.0,0.001346,0.001321,0.005541,0.000953,0.0
2021-04-08,0.002159,0.0,NaN,0.001880,0.000353,0.002267,0.000489,0.000630,0.000638,NaN,...,0.001754,0.002024,0.004675,0.000000,0.0,0.002016,0.001885,0.004776,0.001903,0.0
2021-04-09,0.001030,0.0,NaN,0.000410,-0.000177,0.002345,0.001302,0.000504,0.003189,NaN,...,0.000333,0.002122,0.003723,0.004348,0.0,0.001437,0.001129,0.000731,0.000678,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2021-04-07      0.000000  0.000000     0.0  0.0  0.007576     0.0    0.0   
2021-04-08      0.009804 -0.003322     0.0  0.0  0.007519     0.0    0.0   
2021-04-09      0.000000  0.000000     0.0  0.0  0.000000     0.0    0.0   

Province_State                               ... Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2021-04-07      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2021-04-08      0.000000  0.008264      0.0  ...     0.0    0.0      0.0   
2021-04-09      0.003226  0.000000      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-07          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-08          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-09          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,0.001325,0.001892,0.001061,0.003428,0.005465,0.002671,0.007976,0.004410,0.000293,0.004793,...,0.000625,0.028424,0.002092,9.313226e-10,0.007852,0.003672,0.010210,0.016899,0.002231,0.000536
2021-04-08,0.001278,0.002192,0.001005,0.003574,0.004862,0.002610,0.008821,0.005300,0.000232,0.004963,...,0.000669,0.028662,0.002205,4.656613e-10,0.007404,0.003528,0.010670,0.016969,0.002268,0.001187
2021-04-09,0.001306,0.002218,0.001074,0.003681,0.005330,0.002152,0.009288,0.005221,0.000218,0.004856,...,-0.000211,0.042225,0.002330,2.328306e-10,0.006877,0.004575,0.009873,0.018226,0.002200,0.001876


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,0.001057,0.003706,0.001209,0.008955,0.002452,0.004464,0.003277,0.005085,2.171974e-34,0.003203,...,0.000263,0.035318,0.001313,NaN,0.008157,1.888207e-67,0.006513,0.013727,0.000961,0.001043
2021-04-08,0.001325,0.003162,0.001086,0.008679,0.003054,0.002232,0.004190,0.006381,1.085987e-34,0.003697,...,0.000340,0.035531,0.001446,NaN,0.008477,9.441035e-68,0.008342,0.015991,0.000889,0.000848
2021-04-09,0.001656,0.003105,0.001184,0.004340,0.001527,0.018357,0.004091,0.006184,5.429935e-35,0.003830,...,0.000406,0.052275,0.000723,NaN,0.009762,4.720517e-68,0.009744,0.016960,0.000853,0.001403


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-07,0.000474,0.006244,0.001177,0.003976,0.005495,0.016414,0.004980,0.003892,0.000275,0.006529,...,0.003464,0.026917,0.001885,0.0,0.006432,0.004004,0.006624,0.028526,0.001196,0.000584
2021-04-08,0.000391,0.005889,0.001178,0.005259,0.003026,0.009304,0.005791,0.004235,0.000463,0.006046,...,0.003375,0.025620,0.002033,0.0,0.005774,0.002002,0.007510,0.030170,0.000815,0.000450
2021-04-09,0.000244,0.005769,0.001215,0.004340,0.002255,0.004652,0.006309,0.005237,0.000427,0.006360,...,0.002878,0.027626,0.002117,0.0,0.005636,0.001001,0.007895,0.025619,0.001069,0.000469


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-07,0.000518,0.003149,NaN,0.000790,0.000560,0.000734,0.003316,0.003706,0.002544,1.079975e-117,...,0.001770,0.001359,0.001166,0.005087,0.001638,0.002293,0.003242,0.002334,0.001388,0.001332
2021-04-08,0.000707,0.003347,NaN,0.000791,0.000580,0.000711,0.003654,0.003440,0.003137,5.399876e-118,...,0.001776,0.001477,0.001205,0.008096,0.005233,0.002679,0.003021,0.002712,0.001563,0.001565
2021-04-09,0.000661,0.003618,NaN,0.001164,0.000558,0.000728,0.003869,0.004546,0.003519,2.699938e-118,...,0.001570,0.001322,0.001173,0.007580,0.002617,0.002561,0.003630,0.003010,0.001710,0.001408


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-04-07      0.003371  0.000716  0.000974  0.001919  0.000694  0.000913   
2021-04-08      0.003708  0.000722  0.001604  0.001546  0.000579  0.001282   
2021-04-09      0.002451  0.000653  0.001025  0.000968  0.000521  0.000641   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-07      0.001527  0.000184 -0.000290  0.000873  ...  0.000522   
2021-04-08      0.001473  0.000513  0.000285  0.000987  ...  0.001017   
2021-04-09      0.000500  0.000502  0.000429  0.000494  ...  0.001075   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-07      0.000034 -0.000096  0.004134   0.001854  0.001585  0.001064   
2021-04-08      0.000017  0.000276  0.006901   0.002402  0.001890  0.000532   
2021-04-09      0.000008  0.000138  0.005503   0.001936  0.001903  0.000733   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-04-07           -1.0  0.000141  0.003365  
2021-04-08           -1.0  0.000631  0.001683  
2021-04-09           -1.0  0.002555  0.001624  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-07,0.000519,2.995331e-07,NaN,0.000908,0.001021,0.001609,0.000971,0.000860,0.000275,NaN,...,0.001339,0.001735,0.001081,0.002427,0.000005,0.002358,0.001572,0.004049,0.000829,0.000385
2021-04-08,0.001339,1.497666e-07,NaN,0.001394,0.000687,0.001938,0.000730,0.000745,0.000457,NaN,...,0.001546,0.001880,0.002878,0.001214,0.000002,0.002187,0.001728,0.004412,0.001366,0.000193
2021-04-09,0.001185,7.488328e-08,NaN,0.000902,0.000255,0.002141,0.001016,0.000624,0.001823,NaN,...,0.000940,0.002001,0.003300,0.002781,0.000001,0.001812,0.001428,0.002572,0.001022,0.000096


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga       Baldwin   Barbour          Bibb    Blount   
2021-04-07      0.000947  7.691724e-07  0.000072 -1.935358e-12  0.003907   
2021-04-08      0.005376 -1.660745e-03  0.000036 -9.676788e-13  0.005713   
2021-04-09      0.002688 -8.303725e-04  0.000018 -4.838394e-13  0.002857   

Province_State                                                            ...  \
Admin2           Bullock        Butler   Calhoun  Chambers      Cherokee  ...   
2021-04-07      0.000100  1.488063e-14  0.000269  0.002648  1.453288e-06  ...   
2021-04-08      0.000050  7.440316e-15  0.000135  0.005456  7.266442e-07  ...   
2021-04-09      0.000025  3.720158e-15  0.001680  0.002728  3.633221e-07  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-04-07      5.262344e-07  2.135965e-23  3.974452e-09  1.746012e-29   
2021-04-08      2.631172e-07  1.067983e-23  1.987226e-09  8.730061e-30   
2021-04-09      1.315586e-07  5.339914e-24  9.936129e-10  4.365031e-30   

Province_State                                                    \
Admin2         Sweetwater         Teton         Uinta Unassigned   
2021-04-07       0.000109  5.573599e-17  2.832469e-24  -0.501961   
2021-04-08       0.000054  2.786800e-17  1.416235e-24  -0.501961   
2021-04-09       0.000027  1.393400e-17  7.081173e-25  -0.501961   

Province_State                              
Admin2              Washakie        Weston  
2021-04-07      1.084827e-21  2.303930e-20  
2021-04-08      5.424134e-22  1.151965e-20  
2021-04-09      2.712067e-22  5.759824e-21  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
